In [ ]:
#Reading the file

def read_file(file_path):
    # Function to read the contents of a text file
    with open(file_path, 'r') as file:
        text = file.read()
    return text

txt_file = r"C:\Users\USER\Desktop\algoparams.txt"
txt_content = read_file(txt_file)
#print(txt_content)
json_data = json.loads(txt_content)


def print_keys_and_values(data, indent=0):
    for key, value in data.items():
        if isinstance(value, dict):
            print(f"{' ' * indent}{key}:")
            print_keys_and_values(value, indent + 4)
        else:
            print(f"{' ' * indent}{key}: {value}")

# Print keys and values
print("Keys and values of the JSON data:")
print_keys_and_values(json_data)

In [ ]:
#working for feature handling
import pandas as pd
from sklearn.impute import SimpleImputer
import nltk
from nltk.tokenize import word_tokenize
# Load JSON configuration
nltk.download('punkt')
# Load CSV file
data = pd.read_csv(r"C:\Users\USER\Downloads\DS_Assignment - internship\Screening Test - DS\iris.csv")
feature_handling = json_data['design_state_data']['feature_handling']

target_info = json_data["design_state_data"]["target"]
print(target_info)
tokenized_species=[]
# Example: Print feature handling details for each feature
for feature, details in feature_handling.items():
    a=details["feature_variable_type"]
    if a !="text":
        if details['is_selected']:
            if details['feature_details']['missing_values'] == "Impute":
                print("yes")
                if details['feature_details']['impute_with'] == "Average of values":
                    
                # Impute missing values with mean
                    imputer = SimpleImputer(strategy='mean')
                    data[feature] = imputer.fit_transform(data[[feature]])
                    
                elif details['feature_details']['impute_with'] == "custom":
                # Impute missing values with custom value
                    imputer = SimpleImputer(strategy='constant', fill_value=details['feature_details']['impute_value'])
                    data[feature] = imputer.fit_transform(data[[feature]])
    
    else:
        dummy_species = pd.get_dummies(data['species'])

        data = pd.concat([data, dummy_species], axis=1)

        data.drop(columns=['species'], inplace=True)
        #**We can tokenize for text column but for now i have used One-hotEncoding
        #[for species_text in data['species']:
            #tokens = word_tokenize(species_text)
            tokenized_species.append(tokens)


        #data['tokenized_species'] = tokenized_species]


In [ ]:
#working for Feature Reduction
import pandas as pd
from sklearn.ensemble import RandomForestRegressor



X = data.drop(columns=target_info['target'])  # Assuming 'target_column' is the target column
y = data[target_info['target']]



# Initialize a random forest classifier with specified parameters
num_of_trees = int(json_data['design_state_data']['feature_reduction']['num_of_trees'])
depth_of_trees = int(json_data['design_state_data']['feature_reduction']['depth_of_trees'])
num_of_features_to_keep = int(json_data['design_state_data']['feature_reduction']['num_of_features_to_keep'])

if(json_data["design_state_data"]["target"]["prediction_type"] == "Regression"):
    rf_regressor = RandomForestRegressor(n_estimators=num_of_trees, max_depth=depth_of_trees, random_state=42)

    if(json_data['design_state_data']['feature_reduction']["feature_reduction_method"]=="Tree-based"):

        rf_regressor.fit(X, y)

        feature_importances = rf_regressor.feature_importances_

        sorted_indices = feature_importances.argsort()[::-1]

        
        selected_features = X.columns[sorted_indices[:num_of_features_to_keep]]

        print("Selected Features:for Tree-Based are", selected_features)
    elif(json_data['design_state_data']['feature_reduction']["feature_reduction_method"]=="Principle Component Analysis"):
        pca = PCA(n_components=int(num_of_features_to_keep))

        X_pca = pca.fit_transform(X)
        rf_regressor.fit(X_pca, y)
        print("PCA-Selected Features:")
        print(pca.components_)
    elif(json_data['design_state_data']['feature_reduction']["feature_reduction_method"]=="Correlation with target"):
        correlation_with_target = X.corrwith(y).abs().sort_values(ascending=False)
        selected_features = correlation_with_target.index[:num_of_features_to_keep]
        X_selected = X[selected_features]
        pca = PCA(n_components=num_of_features_to_keep)
        X_pca = pca.fit_transform(X_selected)
        rf_regressor.fit(X_pca, y)

        print("PCA-Selected Features:")
        print(selected_features)
    else:
        print("No reduction")
else:
    print("similar approach for classifier using Random Forest Classifier")




In [ ]:
####WORKING FOR MODELS
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression

target_info = json_data["design_state_data"]["target"]
X = data.drop(columns=[json_data["design_state_data"]["target"]["target"]])  # Features
y = data[json_data["design_state_data"]["target"]["target"]]  # Target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
reg = ["RandomForestRegressor","GradientBoostedRegressor", "LinearRegression"]  # Add other regression models here

csv_file = r"C:\Users\USER\Downloads\DS_Assignment - internship\Screening Test - DS\iris.csv"
data = pd.read_csv(csv_file)

if target_info["prediction_type"] == "Regression":
    # Iterate over each regression model
    for model_class in reg:
        if model_class=="RandomForestRegressor":
            # Extract hyperparameters for Random Forest Regressor
            rf_regressor_params = json_data["design_state_data"]["algorithms"]["RandomForestRegressor"]
            n_estimators = rf_regressor_params["max_trees"]
            max_depth = rf_regressor_params["max_depth"]
            min_samples_leaf_min_value = rf_regressor_params["min_samples_per_leaf_min_value"]
            min_samples_leaf_max_value = rf_regressor_params.get("min_samples_per_leaf_max_value")  # Retrieve max_value, or None if not available
            min_depth = rf_regressor_params["min_depth"]
            max_feature = rf_regressor_params["feature_sampling_statergy"]

            
            model = RandomForestRegressor(
                n_estimators=n_estimators,
                max_depth=max_depth,
                min_samples_leaf=min_samples_leaf_min_value,
                # max_samples_leaf=min_samples_leaf_max_value,
                # min_depth=min_depth ,
                # max_features=default
                # Add other hyperparameters here
            )
            model.fit(X_train, y_train)

            # Evaluate the model
            score = model.score(X_test, y_test)
            print("Model Score (R^2):", score)
        elif model_class=="LinearRegression":
           
            lr_params = json_data["design_state_data"]["algorithms"]["LinearRegression"]
            parallelism = lr_params["parallelism"]
            min_iter = lr_params["min_iter"]
            max_iter = lr_params["max_iter"]
            min_regparam = lr_params["min_regparam"]
            max_regparam = lr_params["max_regparam"]
            min_elasticnet = lr_params["min_elasticnet"]
            max_elasticnet = lr_params["max_elasticnet"]

           
            model = LinearRegression(
                parallelism=parallelism,
                min_iter=min_iter,
                max_iter=max_iter,
                min_regparam=min_regparam,
                max_regparam=max_regparam,
                min_elasticnet=min_elasticnet,
                max_elasticnet=max_elasticnet
                \
            )
            model.fit(X_train, y_train)

            # Evaluate the model
            score = model.score(X_test, y_test)
            print("Model Score (R^2):", score)

        elif model_class=="GradientBoostedRegressor":
            
            gb_regressor_params = json_data["design_state_data"]["algorithms"]["GradientBoostedRegressor"]
            n_estimators = gb_regressor_params["max_iter"]
            learning_rate = gb_regressor_params["learningRate"]
            min_samples_leaf = gb_regressor_params["min_subsample"]
            max_samples_leaf = gb_regressor_params["max_subsample"]
            min_stepsize = gb_regressor_params["min_stepsize"]
            max_stepsize = gb_regressor_params["max_stepsize"]
            min_depth = gb_regressor_params["min_depth"]
            max_depth = gb_regressor_params["max_depth"]

            
            model = GradientBoostedRegressor(
                n_estimators=n_estimators,
                learning_rate=learning_rate,
                min_samples_leaf=min_samples_leaf,
                max_samples_leaf=max_samples_leaf,
                min_stepsize=min_stepsize,
                max_stepsize=max_stepsize,
                min_depth=min_depth,
                max_depth=max_depth
                
            )
            model.fit(X_train, y_train)

            # Evaluate the model
            score = model.score(X_test, y_test)
            print("Model Score (R^2):", score)

        else:
            print("Done with regression model")
